
... ***CURRENTLY UNDER DEVELOPMENT*** ...


### TODO
- GESTIONAR LOS 15 "CASOS" HYCREW PARA EL OUTPUT
- GESTIONAR LOS 15 CASOS HYCREW X 10 SIMULACIONES TESLAKIT PARA EL OUTPUT
- REFACTOR FUNCIONES REPETIDAS HYCREW
- STORE RUNUP OUTPUT


In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# common
import os
import os.path as op

# pip
import numpy as np
import pandas as pd
import xarray as xr
from scipy.interpolate import griddata

# DEV: override installed teslakit
import sys
sys.path.insert(0, op.join(os.path.abspath(''), '..', '..', '..'))

# teslakit
from teslakit.database import Database
from teslakit.rbf import RBF_Interpolation
from teslakit.mda import Normalize



## Database and Site parameters

In [2]:
# --------------------------------------
# Teslakit database

p_data = r'/Users/nico/Projects/TESLA-kit/TeslaKit/data'
db = Database(p_data)

# set site
db.SetSite('ROI')


# reef characteristics
reef_cs = {
    'rslope': 0.0505,
    'bslope': 0.1667,
    'rwidth': 250,
    'cf': 0.0105,
}


# load Hycreww RBF coefficients and sim. variables min. and max.
var_lims, rbf_coeffs = db.Load_HYCREWW()


In [3]:
# hycreww interpolation 

def hycreww_runup(var_lims, rbf_coeffs, dset):
    '''
    Calculates RunUp using hycreww RBFs (level) and linear interpolation (Runup)
    
    var_lims   - hycreww variables min and max limits
    rbf_coeffs - hycreww rbf coefficients
    dset       - input dataset (pandas.DataFrame with "rbf_vns" columns)
    
    '''
    
    # RBF wave conditions 
    rbf_hs = [1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5]
    rbf_hs_lo = [0.005, 0.025, 0.05, 0.005, 0.025, 0.05, 0.005, 0.025, 0.05, 0.005, 0.025, 0.05, 0.005, 0.025, 0.05 ]
    rbf_vns = ['level', 'rslope', 'bslope', 'rwidth', 'cf']
    
    # RBF parameters
    ix_sc = [0, 1, 2, 3, 4]
    ix_dr = []
    minis = [var_lims[x][0] for x in rbf_vns]
    maxis = [var_lims[x][1] for x in rbf_vns]
    
    
    # add reef characteristics
    for p in reef_cs.keys():
        dset[p] = reef_cs[p]

    # discard data outside limits
    for vn in var_lims.keys():
        dset = dset[(dset[vn] > var_lims[vn][0]) &(dset[vn] < var_lims[vn][1])]
    
    
    # RBF dataset to interpolate
    ds_in = dset[rbf_vns]

    # normalize data
    ds_nm ,_ ,_ = Normalize(ds_in.values, ix_sc, ix_dr, minis=minis, maxis=maxis)

    # RBF interpolation (with all cases?)
    ru_out = []
    for rc in rbf_coeffs:
        ro = RBF_Interpolation(rc['constant'], rc['coeff'], rc['nodes'], ds_nm.T)
        ru_out.append(ro)
    ru_z = np.array(ru_out)
    
    # RU Linear interpolation (hs, hs_lo -> runup)
    RU = []
    for c, vn in enumerate(rbf_vns):
        vq = griddata((rbf_hs, rbf_hs_lo), ru_z[:,c], (dset['hs'], dset['hs_lo2']), method='linear')
        RU.append(vq)
        
    # store runup alongside input data
    dset_out = dset.copy()
    dset_out['runup'] = dset_out['level'] + RU[0]
    # TODO: ¿15 RU output? ¿no deberia ser uno tras interpolar? ¿como gestionar esto?
    
    return dset_out



## Hycreww RBF Interpolation: Historical

In [4]:
# Load complete historical data and nearshore waves
data= db.Load_HIST_Complete_daily() 
waves = db.Load_NEARSHORE_RECONSTRUCTION_HIST_storms()

waves = waves.rename_vars({"Hs": "hs", "Tp": "tp", 'Dir':'dir'})  # rename vars
waves['hs_lo2'] = waves['hs']/(1.5613*waves['tp']**2)             # calc. hs_lo2
waves['level'] = data.sel(time=waves.time).level                  # add level


# calculate runup with hycreww
dset = waves[['hs', 'tp', 'dir', 'level', 'hs_lo2']].to_dataframe()
out = hycreww_runup(var_lims, rbf_coeffs, dset)
print(out)


                  hs        tp         dir     level    hs_lo2  rslope  \
time                                                                     
1996-12-09  1.006710  4.752300  158.311005  0.628119  0.028550  0.0505   
1999-08-11  1.307120  5.431180  135.128006  0.035807  0.028382  0.0505   
2005-08-24  1.021310  4.801500   92.919403  0.001576  0.028374  0.0505   
2005-11-11  1.387720  5.902350  100.340004  0.863839  0.025513  0.0505   
2013-10-14  1.630730  7.990180  133.785004  0.834181  0.016360  0.0505   
...              ...       ...         ...       ...       ...     ...   
2013-12-21  1.421666  8.856265   87.065544 -0.047445  0.011609  0.0505   
2013-12-26  2.228316  8.585747   75.318932  0.702555  0.019361  0.0505   
2013-12-28  2.480173  8.736678   70.589622  0.946555  0.020811  0.0505   
2013-12-29  2.178743  7.840980   72.677254  0.920555  0.022698  0.0505   
2014-01-06  1.988687  8.287806   61.564789  0.149555  0.018544  0.0505   

            bslope  rwidth      cf   


## Hycreww RBF Interpolation: Simulation

In [ ]:
# Load complete simulations data and nearshore waves

n_sims_DWT = 10
data_list = db.Load_SIM_Complete_storms(n_sims=n_sims_DWT)
waves_list = db.Load_NEARSHORE_RECONSTRUCTION_SIM_storms(n_sims=n_sims_DWT)

# iterate storms waves simulations
for dd, ww in zip(data_list, waves_list):
    
    ww = ww.rename_vars({"Hs": "hs", "Tp": "tp", 'Dir':'dir'})  # rename vars
    ww['hs_lo2'] = ww['hs']/(1.5613*ww['tp']**2)                # calc. hs_lo2
    ww['level'] = dd.sel(time=ww.time).level                    # add level
    

    # calculate runup with hycreww
    dset = ww[['hs', 'tp', 'dir', 'level', 'hs_lo2']].to_dataframe()
    out = hycreww_runup(var_lims, rbf_coeffs, dset)
    print(out)
    print()

